In [ ]:
f = x -> (x+1)^2*cos((2*x+1)/(x-4.3));

using QuadGK
@show I,errest = quadgk(f,0,4,atol=1e-14,rtol=1e-14);  # 'exact' value

We perform the integration and show the nodes selected underneath the curve. 

In [ ]:
include("../FNC.jl")
Q,t = FNC.intadapt(f,0,4,0.001)
@show num_nodes = length(t);

using Plots
plot(f,0,4,color=:black,legend=:none,
    xlabel="x",ylabel="f(x)",title="Adaptive node selection")
plot!(t,f.(t),seriestype=:sticks,m=(:o,2))

The error turns out to be a bit more than we requested. It's only an estimate, not a guarantee.

In [ ]:
@show err = I - Q;

Let's see how the number of integrand evaluations and the error vary with the requested tolerance.

In [ ]:
tol_ = @. 10.0^(-4:-1:-14)
err_ = zeros(size(tol_))
num_ = zeros(Int,size(tol_))
for i = 1:length(tol_)
    Q,t = FNC.intadapt(f,0,4,tol_[i])
    err_[i] = I - Q
    num_[i] = length(t)
end

using DataFrames
DataFrame(tol=tol_,error=err_,f_evals=num_)

As you can see, even though the errors are not less than the estimates, the two columns decrease in tandem. If we consider now the convergence not in $h$ (which is poorly defined) but in the number of nodes actually chosen, we come close to the fourth order accuracy of the underlying Simpson scheme.

In [ ]:
plot(num_,abs.(err_),m=:o,label="results",
    xaxis=(:log10,"number of nodes"),yaxis=(:log10,"error"),
    title="Convergence of adaptive quadrature")
order4 = @. 0.01*(num_/num_[1])^(-4)
plot!(num_,order4,l=:dash,label="\$O(n^{-4})\$")